somatic değişkenini en iyi açıklayan değişkenler hangileri? 

In [1]:
import numpy as np 
import pandas as pd 

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso 
from sklearn.preprocessing import StandardScaler 
from sklearn.pipeline import Pipeline

import matplotlib.pyplot as plt 

df = pd.read_csv("final_learning_potential_dataset.csv")

print(df.shape)
df.head()

(427, 7)


,stress_congeneric_score,control_congeneric,self_efficacy_congeneric,depression_congeneric_score,somatic_congeneric,cog_affect_congeneric,learning_potential_score
0,-0.994499,0.568381,1.568341,-0.852725,-1.522905,-0.294627,2.153331
1,-1.601830,1.258011,1.846971,-1.430514,-1.403349,-1.390766,3.369578
2,0.183967,-0.516081,0.397377,-0.347319,0.019725,-0.594503,0.234547
3,0.419527,-0.581236,-0.063644,-0.825759,-1.003940,-0.594503,0.534051
4,-1.093718,1.413048,0.214986,0.870991,1.372580,0.501635,-0.020603


predictors ve target seçimi 

In [2]:
target_col = "somatic_congeneric"

predictor_cols=  [
    "stress_congeneric_score",
    "control_congeneric",
    "self_efficacy_congeneric",
    "depression_congeneric_score",
    "cog_affect_congeneric"
]

6 değişkenli PCA ile hesaplandığı için data leakage sebebiyle learning_potential_score, predictors bünyesine alınmadı 

In [3]:
model_df = df[[target_col] + predictor_cols].dropna()

X = model_df[predictor_cols]
y = model_df[target_col]

X.head(), y.head()
X.shape, y.shape

((427, 5), (427,))

In [ ]:
# train test ayrımı 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((341, 5), (86, 5), (341,), (86,))

In [7]:
from sklearn.metrics import r2_score, mean_squared_error

ols_model = LinearRegression()

ols_model.fit(X_train, y_train)

y_pred = ols_model.predict(X_test)

r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

r2, mse


(0.9969829171258514, 0.003423983253678735)

In [8]:
coeff_df = pd.DataFrame({
    'predictor': predictor_cols,
    'coefficient': ols_model.coef_
})

coeff_df


,predictor,coefficient
0,stress_congeneric_score,-0.121481
1,control_congeneric,-0.089748
2,self_efficacy_congeneric,-0.039370
3,depression_congeneric_score,2.163663
4,cog_affect_congeneric,-1.234639


OLS MODELİ KATSAYI ANALİZİ VE İLK BULGULAR 

Özellikle 2 değişkenin katsayısının çok yüksek olduğu görülmüştür: 
- **depression_congeneric_score → β = +2.16**
- **cog_affect_congeneric → β = –1.23**

Tüm değişkenler ve hedef z-score formatında olduğundan, bu katsayılar **standardize edilmiş etki büyüklüğü** olarak yorumlanabilir. Bu açıdan:
-depresyon puanındaki 1 standart sapmalık artış, somatik şikayetlerde 2.16 standart sapmalık bir artış ile, 
-kognitif efektif puanındaki 1 standart sapmalık artış ise somatik şikayetlerde 1.23 standart sapmalık bir azalma ile ilikşilendirilir.

Bu büyüklükteki katsasyılar psikoloji literatüründe çok güçlü etki kabul edilmektedir 

Bu bulgu 2 noktayı işaret etmektedir:

1- **Shared Variance /*Yüksek Ortak Varyans***
Tüm congeneric skorlar aynı kişiden, aynı oturumda benzer psikolojik içerklerle ölçülmüştür. Bu nedenle depresyon ve kognitif efekt ile somatik arasında yüksek derecede örtüşen psikolojik yapı bulunmaktadır. 

2- **Multicollinearity /*Çoklu Doğrusal Bağlantı***
Predictor'ların birbirine yüksek derecede benzediği durumlarda OLS katsayıları şişebilir, model narin hale gelebilir ve katsayılar gerçek etkilerinden sapabilir. 

Bu sebeple OLS modeli yalnızca baseline olarak değerlendirilmiş; katsayıların stabilitesini test etmek ve multicollinearity etkisini azaltmak için **Ridge** ve **Lasso** regresyon modellerine geçilmiştir. 